In [ ]:
!pip install -r requirements.txt

In [ ]:
# Imports modules for socket operations to connect to the internet and download data
import socket,warnings
import os
try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in top right and set 'Internet' switch to on")

# Step 1: Download images of birds and non-birds

In [ ]:
# Imports modules for searching and retrieving images
from duckduckgo_search import ddg_images
from fastcore.all import *

# Function to search and retrieve images using DuckDuckGo and return the image URLs
def search_images(term, max_images=200): return L(ddg_images(term, max_results=max_images)).itemgot('image')

In [ ]:
# Searches 1 bird photo image and retrieves its url 
urls = search_images('bird photos', max_images=1)
urls[0]

In [ ]:
# Imports the 'download_url' function
from fastdownload import download_url

# Download destination test image from the URL
dest = 'bird.jpg'
download_url(urls[0], dest, show_progress=False)

# Imports modules for image processing
from fastai.vision.all import *

# Opens and formats image
im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
# Searches for forest photos
download_url(search_images('forest photos', max_images=1)[0], 'forest.jpg', show_progress=False)

# Opens the image
Image.open('forest.jpg').to_thumb(256,256)

In [ ]:
# Defines a list of terms for the images (training data)
searches = 'forest','bird'

path = Path('bird_or_not')
from time import sleep

# Loops through each search term and downloads images related to it with certain characteristics
for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    sleep(10)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{o} sun photo'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} shade photo'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)

# Step 2: Train our model

In [ ]:
# Remove any failed images from our data
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
# Defines a DataBlock for loading the data
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

# Displays batch of data
dls.show_batch(max_n=6)

In [ ]:
# Creates the resnet model
learn = vision_learner(dls, resnet18, metrics=error_rate)

# Trains the model
learn.fine_tune(3)

# Step 3: Use the model

In [ ]:
# Outputs prediction for our test image
is_bird,_,probs = learn.predict(PILImage.create('bird.jpg'))
print(f"This is a: {is_bird}.")
print(f"Probability it's a bird: {probs[0]:.4f}")